In [ ]:
from karabo.simulation.signal.plotting import SignalPlotting, SegmentationPlotting
from karabo.simulation.signal.signal_21_cm import Signal21cm
from karabo.simulation.signal.synchroton_signal import SignalSynchroton
from karabo.simulation.signal.galactic_foreground import SignalGalacticForeground
from karabo.simulation.signal.superimpose import Superimpose
from karabo.simulation.signal.superpixel_segmentation import SuperpixelSegmentation
from karabo.simulation.signal.seg_u_net_segmentation import SegUNetSegmentation

from astropy.coordinates import Angle, SkyCoord
from astropy import units
from karabo.simulation.signal.typing import Image2D, Image3D


In [ ]:
print(SignalGalacticForeground.available_redshifts())
print("- " * 3)
print(Signal21cm.available_redshifts())


**7.6** lower:7.57 (0.03) higher:7.664 (0.064)  
**8.4** lower:8.397 (0.003) higher:8.515 (0.115)  
**9.2** lower:9.164 (0.036) higher:9.308 (0.108)  
**9.9** lower:9.771 (0.129) higher:9.938 (0.038)

**10.7** lower:10.673 (0.027) higher:10.877 (0.177)  
**11.5** lower:11.313 (0.187) higher:11.546 (0.046)  
**12.2** lower:12.048 (0.152) higher:12.318 (0.118)  
**13.0** lower:12.903 (0.097) higher:13.221 (0.221)

**14.3** lower:14.294 (0.006) higher:14.699 (0.399)  
**15.1** lower:14.699 (0.401) higher:15.132 (0.032)  
**15.8** lower:15.596 (0.204) higher:16.095 (0.295)  
**16.6** lower:16.095 (0.505) higher:16.633 (0.033)

**17.4** lower:17.215 (0.185) higher:17.848 (0.448)  
**18.1** lower:17.848 (0.252) higher:18.54 (0.44)  
**18.9** lower:18.54 (0.36) higher:19.298 (0.398)  
**19.7** lower:19.298 (0.402) higher:20.134 (0.434)

**20.4** lower:20.134 (0.266) higher: -  
**21.2** lower: - higher: -  
**22.0** lower: - higher: -  
**22** lower: - higher: -


**7.6** lower:7.57 (0.03)  
**8.4 lower:8.397 (0.003)**  
**9.2** lower:9.164 (0.036)  
**9.9** higher:9.938 (0.038)

**10.7 lower:10.673 (0.027)**  
**11.5** higher:11.546 (0.046)  
**13.0** lower:12.903 (0.097)

**14.3 lower:14.294 (0.006)**  
**15.1** higher:15.132 (0.032)  
**16.6** higher:16.633 (0.033)

**17.4 lower:17.215 (0.185)**  
**18.1** lower:17.848 (0.252)  
**18.9** lower:18.54 (0.36)

**20.4 lower:20.134 (0.266)**


In [ ]:
# (Galactic, Signal21)
redshift_sig21 = [
    8.397,
    10.673,
    14.294,
    17.215,
    20.134,
]
redshift_gf = [
    8.4,
    10.7,
    14.3,
    17.4,
    20.4,
]


In [ ]:
from karabo.simulation.signal.base_signal import BaseSignal
from karabo.simulation.signal.typing import BaseImage


class ZPipeline:
    def __init__(
        self, redshifts_sig21: list[float], redshifts_galactic: list[float]
    ) -> None:
        self.redshifts_sig21 = redshifts_sig21
        self.redshifts_galactic = redshifts_galactic

        # prepare signal21cm
        files_sig21 = [
            Signal21cm.get_xfrac_dens_file(z=z, box_dims=244 / 0.7)
            for z in redshifts_sig21
        ]
        self.signal_21 = Signal21cm(files_sig21)
        grid_size = (250, 250)

        # prepare galactic foreground
        cent = SkyCoord(ra=10 * units.degree, dec=20 * units.degree, frame="icrs")
        fov = Angle([20, 20], unit=units.degree)
        self.gf = SignalGalacticForeground(
            cent,
            redshifts=redshifts_galactic,
            fov=fov,
            grid_size=grid_size,
        )

        # prepare synchroton
        self.sync = SignalSynchroton(
            centre=cent,
            fov=fov,
            grid_size=grid_size,
        )

    def run(self) -> list[BaseImage]:
        image_sync = self.sync.simulate()[0]
        images_sig21 = self.signal_21.simulate()
        images_galactic = self.gf.simulate()

        images = []
        for im_21, im_gal in zip(images_sig21, images_galactic):
            image = Superimpose.combine(im_21, im_gal, image_sync)
            images.append(image)

        res = []
        for image, sig_21 in zip(images, images_sig21):
            superpixel_image = SuperpixelSegmentation().segment(image)
            _ = SegmentationPlotting.superpixel_plotting(superpixel_image, sig_21)

            seg_u_net_image = SegUNetSegmentation(tta=0).segment(image)
            _ = SegmentationPlotting.seg_u_net_plotting(seg_u_net_image)

            res.append((superpixel_image, seg_u_net_image))

        return res


images = ZPipeline(redshifts_sig21=redshift_sig21, redshifts_galactic=redshift_gf).run()


In [ ]:
import multiprocessing
import threading
from karabo.simulation.signal.base_signal import BaseSignal
from karabo.simulation.signal.typing import BaseImage
import asyncio

import nest_asyncio  # ONLY for IPYTHON

nest_asyncio.apply()  # ONLY for IPYTHON


class ThreadWithReturnValue(threading.Thread):
    def __init__(
        self, group=None, target=None, name=None, args=(), kwargs={}, Verbose=None
    ):
        threading.Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args, **self._kwargs)

    def join(self, *args):
        threading.Thread.join(self, *args)
        return self._return


class ZPipeline:
    def __init__(
        self, redshifts_sig21: list[float], redshifts_galactic: list[float]
    ) -> None:
        self.redshifts_sig21 = redshifts_sig21
        self.redshifts_galactic = redshifts_galactic

        # prepare signal21cm
        files_sig21 = [
            Signal21cm.get_xfrac_dens_file(z=z, box_dims=244 / 0.7)
            for z in redshifts_sig21
        ]
        self.signal_21 = Signal21cm(files_sig21)
        grid_size = (250, 250)

        # prepare galactic foreground
        cent = SkyCoord(ra=10 * units.degree, dec=20 * units.degree, frame="icrs")
        fov = Angle([20, 20], unit=units.degree)
        self.gf = SignalGalacticForeground(
            cent,
            redshifts=redshifts_galactic,
            fov=fov,
            grid_size=grid_size,
        )

        # prepare synchroton
        self.sync = SignalSynchroton(
            centre=cent,
            fov=fov,
            grid_size=grid_size,
        )

        self._results = multiprocessing.Queue()

    def run(self) -> list[BaseImage]:
        sig_sync_thread = ThreadWithReturnValue(target=self._simulate_syncroton)
        sig_21cm_thread = ThreadWithReturnValue(target=self._simulate_signal_21cm)
        sig_gf_thread = ThreadWithReturnValue(target=self._simulate_galactic_foreground)

        sig_sync_thread.start()
        sig_21cm_thread.start()
        sig_gf_thread.start()

        image_sync = sig_sync_thread.join()[0]
        images_galactic = sig_21cm_thread.join()
        images_sig21 = sig_gf_thread.join()

        print("Done simulating")

        images: list[BaseImage] = []
        for im_21, im_gal in zip(images_sig21, images_galactic):
            image = Superimpose.combine(im_21, im_gal, image_sync)
            images.append(image)

        print("Done combining")

        procs: list[multiprocessing.Process] = []
        for idx, (image, sig_21) in enumerate(zip(images, images_sig21)):
            print(f"Segmenting {idx}")

            su_proc = multiprocessing.Process(
                target=self._run_superpixel,
                args=(image, idx),
            )
            su_proc.start()
            procs.append(su_proc)

            segu_proc = multiprocessing.Process(
                target=self._run_segunet,
                args=(image, idx),
            )
            segu_proc.start()
            procs.append(segu_proc)

        for proc in procs:
            proc.join()

        return [x[1] for x in sorted(self._results, key=lambda x: x[0])]

    def _simulate_syncroton(self) -> Image2D:
        print("Running synchroton")
        return self.sync.simulate()

    def _simulate_signal_21cm(self) -> Image2D:
        print("Running 21cm")
        return self.signal_21.simulate()

    def _simulate_galactic_foreground(self) -> Image2D:
        print("Running Galactic foreground")
        return self.gf.simulate()

    def _run_superpixel(self, image: Image3D, idx: int) -> None:
        print("Running Superpixel")
        segmented = SuperpixelSegmentation().segment(image)
        self._results.put((idx, segmented))

    def _run_segunet(self, image: Image3D, idx: int) -> None:
        print("Running Superpixel")
        segmented = SegUNetSegmentation(tta=0).segment(image)
        self._results.put((idx, segmented))


images = ZPipeline(redshifts_sig21=redshift_sig21, redshifts_galactic=redshift_gf).run()


In [ ]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f


async def main():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)


asyncio.run(main())


In [ ]:
_ = SignalPlotting.brightness_temperature(sig_21)
_ = SignalPlotting.general_img(gf_sim, "Galactic foreground")  # TODO
_ = SignalPlotting.general_img(sync_sim, "Synchroton")

out_img = Image2D(
    data=out.data[0, :, :],
    x_label=out.x_label[0, 0, :],
    y_label=out.y_label[0, 0, :],
    redshift=out.redshift,
    box_dims=out.box_dims,
)
_ = SignalPlotting.general_img(
    out_img, "Combined signals", x_label="...", y_label="..."
)
